In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import emoji # a esta para buscar emojis en las reseñas
from unidecode import unidecode

C:\Users\kanza\AppData\Local\Temp\ipykernel_10060\390483211.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Importamos la libería nltk para hacer el análisis de sentimiento de las reseñas.

In [2]:
#vader_lexicon es un conjunto de datos de palabras con valores puntuaciones y polaridad que se utiliza para el análisis.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kanza\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

Cargamos los dataframes

In [4]:
user_reviews = pd.read_csv('../user_reviews_df_cleaned.csv')
games = pd.read_csv('../dfSteamCleaned.csv')

C:\Users\kanza\AppData\Local\Temp\ipykernel_10060\2731448086.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv('../dfSteamCleaned.csv')


Nos quedamos solo con las columnas que necesitamos para esta función

In [5]:
user_reviews = user_reviews[['item_id','sentiment_analysis','recommend']]
user_reviews

,item_id,sentiment_analysis,recommend
0,1250.0,2,True
1,22200.0,2,True
2,43110.0,2,True
3,251610.0,2,True
4,227300.0,2,True
...,...,...,...
59328,70.0,2,True
59329,362890.0,2,True
59330,273110.0,2,True
59331,730.0,1,True


In [6]:
games = games[['id','developer','year']]

In [7]:
games

,id,developer,year
0,NaN,NaN,0.0
1,NaN,NaN,0.0
2,NaN,NaN,0.0
3,NaN,NaN,0.0
4,NaN,NaN,0.0
...,...,...,...
120439,773640.0,"Nikita ""Ghost_RUS""",2018.0
120440,733530.0,Sacada,2018.0
120441,610660.0,Laush Dmitriy Sergeevich,2018.0
120442,658870.0,"xropi,stev3ns",2017.0


El análisis de sentimiento no es muy exacto por varias razones, como ser, errores de ortografía, falta de signos de puntuación, reseñas escritas en otro idioma, palabras con valoración negativa pero descontextualizadas o el uso de caractéres no alanuméricos como emojis, que normalmente se utlizan para censurar malas palabras o contenido ofensivo. Debido a esto el análisis de sentimiento puede ser erróneo y presentar una discordancia entre el campo recommend y el valor del sentimiento ya que, normalmente, uno esperaría que si un usuario recomienda un juego tenga una reseña positiva sobre el mismo.

También puede suceder que el usuario recomiende el juego pero en la reseña escriba una queja sobre algún aspecto en particular del juego, y de esa forma el analizador de sentimientos lo califique como negativo pero aún así el juego siga siendo del agrado del usuario.

Por las cuestiones recién mencionadas, considero que deberíamos guiarnos del campo recommend para hacer la valoración del usuario sobre los items ya que este campo, al ser booleano, tiene un margen de error mucho menor que el análisis de sentimientos.
Pero para cumplir con las consignas de la función, utilizaremos la combinación requerida (recommend==True y sentiment==Pos o Neu).

Con el análisis de sentimientos hecho, uniremos los dataframes según el id del juego para obtener el desarrollador y el año de lanzamiento.

In [8]:
df = pd.merge(user_reviews, games, left_on='item_id', right_on='id')
df

,item_id,sentiment_analysis,recommend,id,developer,year
0,1250.0,2,True,1250.0,Tripwire Interactive,2009.0
1,22200.0,2,True,22200.0,ACE Team,2009.0
2,227300.0,2,True,227300.0,SCS Software,2013.0
3,239030.0,2,True,239030.0,3909,2013.0
4,248820.0,2,True,248820.0,"Hopoo Games, LLC",2013.0
...,...,...,...,...,...,...
2526691,70.0,2,True,70.0,Valve,1998.0
2526692,362890.0,2,True,362890.0,Crowbar Collective,2015.0
2526693,273110.0,2,True,273110.0,Nexon,2014.0
2526694,730.0,1,True,730.0,Valve,2012.0


In [9]:
df = df[(df['recommend']==True) & ((df['sentiment_analysis']==2) | (df['sentiment_analysis']==1))]

In [10]:
df

,item_id,sentiment_analysis,recommend,id,developer,year
0,1250.0,2,True,1250.0,Tripwire Interactive,2009.0
1,22200.0,2,True,22200.0,ACE Team,2009.0
2,227300.0,2,True,227300.0,SCS Software,2013.0
3,239030.0,2,True,239030.0,3909,2013.0
4,248820.0,2,True,248820.0,"Hopoo Games, LLC",2013.0
...,...,...,...,...,...,...
2526691,70.0,2,True,70.0,Valve,1998.0
2526692,362890.0,2,True,362890.0,Crowbar Collective,2015.0
2526693,273110.0,2,True,273110.0,Nexon,2014.0
2526694,730.0,1,True,730.0,Valve,2012.0


Ya no necesitamos la columna item_id

In [11]:
df.drop(columns=['item_id'], inplace=True)

C:\Users\kanza\AppData\Local\Temp\ipykernel_10060\3317028349.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['item_id'], inplace=True)


In [12]:
df

,sentiment_analysis,recommend,id,developer,year
0,2,True,1250.0,Tripwire Interactive,2009.0
1,2,True,22200.0,ACE Team,2009.0
2,2,True,227300.0,SCS Software,2013.0
3,2,True,239030.0,3909,2013.0
4,2,True,248820.0,"Hopoo Games, LLC",2013.0
...,...,...,...,...,...
2526691,2,True,70.0,Valve,1998.0
2526692,2,True,362890.0,Crowbar Collective,2015.0
2526693,2,True,273110.0,Nexon,2014.0
2526694,1,True,730.0,Valve,2012.0


In [13]:
df = df.groupby(['id','developer','year'])['recommend'].sum().reset_index()
df

,id,developer,year,recommend
0,10.0,Valve,2000.0,55
1,20.0,Valve,1999.0,11
2,30.0,Valve,2003.0,3
3,40.0,Valve,2001.0,True
4,50.0,Gearbox Software,1999.0,3
...,...,...,...,...
2610,520550.0,SCS Software,2016.0,True
2611,521340.0,Vladimir Maslov,2016.0,2
2612,521430.0,David Mulder,2016.0,True
2613,521570.0,Tamationgames,2016.0,True


In [14]:
df = df.sort_values(by=['year', 'recommend'], ascending=[False, False])
df

,id,developer,year,recommend
1639,304930.0,Smartly Dressed Games,2017.0,851
1610,301520.0,Freejam,2017.0,229
1981,346110.0,"Studio Wildcard,Instinct Games,Efecto Studios,...",2017.0,197
1516,291550.0,Blue Mammoth Games,2017.0,68
962,236110.0,Trendy Entertainment,2017.0,37
...,...,...,...,...
306,32340.0,LucasArts,1990.0,True
1840,327950.0,"MicroProse Software, Inc",1989.0,True
591,202170.0,United Front Games,0.0,48
837,223850.0,Futuremark,0.0,2


In [15]:
top3 = df.groupby('year').head(3)
top3

,id,developer,year,recommend
1639,304930.0,Smartly Dressed Games,2017.0,851
1610,301520.0,Freejam,2017.0,229
1981,346110.0,"Studio Wildcard,Instinct Games,Efecto Studios,...",2017.0,197
707,211820.0,Chucklefish,2016.0,310
883,227940.0,Reto-Moto,2016.0,145
...,...,...,...,...
306,32340.0,LucasArts,1990.0,True
1840,327950.0,"MicroProse Software, Inc",1989.0,True
591,202170.0,United Front Games,0.0,48
837,223850.0,Futuremark,0.0,2


In [16]:
top3['year'] = top3['year'].astype(int)

C:\Users\kanza\AppData\Local\Temp\ipykernel_10060\128510206.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top3['year'] = top3['year'].astype(int)


In [17]:
top3

,id,developer,year,recommend
1639,304930.0,Smartly Dressed Games,2017,851
1610,301520.0,Freejam,2017,229
1981,346110.0,"Studio Wildcard,Instinct Games,Efecto Studios,...",2017,197
707,211820.0,Chucklefish,2016,310
883,227940.0,Reto-Moto,2016,145
...,...,...,...,...
306,32340.0,LucasArts,1990,True
1840,327950.0,"MicroProse Software, Inc",1989,True
591,202170.0,United Front Games,0,48
837,223850.0,Futuremark,0,2


In [18]:
top3.to_csv('bestDeveloperResults.csv', index=False)